In [14]:
using Dojo
using DojoEnvironments
using LinearAlgebra
using Rotations


In [15]:
HORIZON = 2000
quadrotor_env = get_environment(:quadrotor_waypoint; horizon=HORIZON)




DojoEnvironments.QuadrotorWaypoint{Float64, 2000}

In [16]:
ref_position_xyz_world = [0;0;1]
next_waypoint = 1
state_traj = zeros(size(get_state(quadrotor_env))[1], HORIZON)


12×2000 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [17]:

reference_traj = zeros(4, HORIZON)
plot_dict = Dict("state_traj" => state_traj, "reference_traj" => reference_traj)

Dict{String, Matrix{Float64}} with 2 entries:
  "state_traj"     => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 …
  "reference_traj" => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;…

In [18]:
function get_transformation_body_to_world(x, y, z, roll, pitch, yaw)
    R = RotXYZ(roll, pitch, yaw)
    R_matrix = convert(Array{Float64, 2}, R) # Convert to matrix
    
    position_w = [x, y, z]
    T_world_to_body = [R_matrix' -R_matrix' * position_w; 0 0 0 1]
    return T_world_to_body
end

get_transformation_body_to_world (generic function with 1 method)

In [19]:
function MMA!(roll, pitch, yaw, thrust)
    u = zeros(4)
    u[1] = thrust + roll - pitch + yaw
    u[2] = thrust + roll + pitch - yaw
    u[3] = thrust - roll + pitch + yaw
    u[4] = thrust - roll - pitch - yaw
    return u

end


MMA! (generic function with 1 method)

In [20]:
function sensing_and_estiamtion(environment)

    state = get_state(environment)
    position = state[1:3] # x,y,z
    orientation = state[4:6] # axis*angle
    linear_velocity = state[7:9] # vx, vy, vz
    angular_velocity = state[10:12] # ωx, ωy, ωz 
    
    altitude = position[3]
    roll, pitch, yaw = orientation
    
    return roll, pitch, yaw, altitude
end
    


sensing_and_estiamtion (generic function with 1 method)

In [21]:
function position_to_quadrotor_oritentation_controller(environment, k)
    global ref_position_xyz_world
    position = get_state(environment)[1:3]
    roll, pitch, yaw = get_state(environment)[4:6]
    linear_velocity = get_state(environment)[7:9]
    
    
   transformation_b_w = get_transformation_body_to_world(position[1], position[2], position[3], roll, pitch, yaw) 
   println("transformation_b_w: ", transformation_b_w)
   ref_position_b = transformation_b_w * [ref_position_xyz_world;1]
   ref_position_b = ref_position_b[1:3]
   ref_position_b = ref_position_xyz_world
   println("current position: , $position")
   plot_dict["state_traj"][:, k] = get_state(environment)
   println("ref_position_b: ", ref_position_b)  
   
   K_p_xy_roll = K_p_xy_pitch = 0.04
   K_d_xy_roll = K_d_xy_pitch = 0.1
   
   roll_ref = -(K_p_xy_roll * (ref_position_b[2] - position[2]) + K_d_xy_roll * (0 - linear_velocity[2]))
   pitch_ref = K_p_xy_pitch * (ref_position_b[1] - position[1]) + K_d_xy_pitch * (0 - linear_velocity[1])
   plot_dict["reference_traj"][1:3, k] = [roll_ref, pitch_ref, ref_position_b[2]-position[2]]
   
   return roll_ref, pitch_ref
end
   


position_to_quadrotor_oritentation_controller (generic function with 1 method)

In [27]:
function cascade_controller!(environment, k)
    global ref_position_xyz_world
    # get the current system values
    roll, pitch, yaw, altitude = sensing_and_estiamtion(environment)
    println("states: $(sensing_and_estiamtion(environment))")
    # set some reference - TODO - make this to be set from outside or smth
    #roll_ref, pitch_ref, yaw_ref, altitude_ref = [0.0, 0.0, 1.57, 1]
    roll_ref, pitch_ref, = position_to_quadrotor_oritentation_controller(environment, k)
    println("roll ref: $roll_ref, pitch_ref: $pitch_ref")
    yaw_ref = 0 #1.57
    altitude_ref = ref_position_xyz_world[3] #1

    # PIDs
    # roll
    K_p_roll = 4 #1
    K_d_roll = 1
    # pitch
    K_p_pitch = 4 #1
    K_d_pitch = 1
    # yaw
    K_p_yaw = 4 #1
    K_d_yaw = 1
    # thrust
    K_p_thrust = 10 # kinda tuned
    K_d_thrust = 10 # kinda tuned
    # thrust feedforward is there to compensate gravity and i took the numbers from the cascaded hover example.
    thrust_feedforward = 20 * 5.1 * 1/sqrt(4)#normalize([1;1;1;1])

    roll_cntrl = K_p_roll * (roll_ref - roll) + K_d_roll * (0 - get_state(environment)[10])
    pitch_cntrl = K_p_pitch * (pitch_ref - pitch) + K_d_pitch * (0 - get_state(environment)[11])
    yaw_cntrl = K_p_yaw * (yaw_ref - yaw) + K_d_yaw * (0 - get_state(environment)[12])
    thrust = K_p_thrust * (altitude_ref - altitude) + K_d_thrust * (0 - get_state(environment)[9])+ thrust_feedforward

    u = MMA!(roll_cntrl, pitch_cntrl, yaw_cntrl, thrust)
    set_input!(environment, u)

end



cascade_controller! (generic function with 1 method)

In [23]:

function controller!(environment, k)
    println("k: ", k) #k = runs 
    fly_through_waypoints_controller!(environment, k)
end



controller! (generic function with 1 method)

In [24]:
function default_controller!(environment, k)
    set_input!(environment, rotor_speeds)
end

default_controller! (generic function with 1 method)

In [25]:
function fly_through_waypoints_controller!(environment, k)
    global next_waypoint
    global ref_position_xyz_world
    waypoints = 
    [
        [1;1;0.3],
        [2;0;0.3],
        [1;-1;0.3],
        [0;0;0.3],
    ]
    if norm(get_state(environment)[1:3]-waypoints[next_waypoint]) < 1e-1
        if next_waypoint < 4
            println("next_waypoint: ", next_waypoint)
            next_waypoint += 1
        end
    end
    ref_position_xyz_world = waypoints[next_waypoint]
    println("ref_position_xyz_world in waypoint funciton: ", ref_position_xyz_world)
    cascade_controller!(environment, k)
end

fly_through_waypoints_controller! (generic function with 1 method)

In [28]:
# Simulate
initialize!(quadrotor_env, :quadrotor)
simulate!(quadrotor_env, controller!; record=true)


vis = visualize(quadrotor_env)
render(vis)


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), UInt8[0x83, 0xa7, 0x6f, 0x70, 0x74, 0x69, 0x6f, 0x6e, 0x73, 0x82  …  0x5f, 0x61, 0x6e, 0x69, 0x6d, 0x61, 0x74, 0x69, 0x6f, 0x6e], Dict{String, MeshCat.SceneTrees.SceneNode}("Background" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}("top_color" => [0x84, 0xa8, 0x70, 0x72, 0x6f, 0x70, 0x65, 0x72, 0x74, 0x79  …  0x42, 0x61, 0x63, 0x6b, 0x67, 0x72, 0x6f, 0x75, 0x6e, 0x64], "bottom_color" => [0x84, 0xa8, 0x70, 0x72, 0x6f, 0x70, 0x65, 0x72, 0x74, 0x79  …  0x42, 0x61, 0x63, 0x6b, 0x67, 0x72, 0x6f, 0x75, 0x6e, 0x64]), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "Grid" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}("visible" => [0x84, 0xa8, 0x70, 0x72, 0x6f, 0x70, 0x65, 0x72, 0x74, 0x79  …  0x70, 0x61, 0x74, 0x68, 0xa5, 0x2f, 0x47, 0x72, 0x69, 0x64]), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("waypoints" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("waypoint1" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x77, 0x61, 0x79, 0x70, 0x6f, 0x69, 0x6e, 0x74, 0x31], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "waypoint3" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x77, 0x61, 0x79, 0x70, 0x6f, 0x69, 0x6e, 0x74, 0x33], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "waypoint4" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x77, 0x61, 0x79, 0x70, 0x6f, 0x69, 0x6e, 0x74, 0x34], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "waypoint2" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x77, 0x61, 0x79, 0x70, 0x6f, 0x69, 0x6e, 0x74, 0x32], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "floor" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x68, 0x63, 0x61, 0x74, 0x2f, 0x66, 0x6c, 0x6f, 0x6f, 0x72], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x68, 0x63, 0x61, 0x74, 0x2f, 0x66, 0x6c, 0x6f, 0x6f, 0x72], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "robot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("bodies" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("rotor_3__id_1" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x6f, 0x72, 0x5f, 0x33, 0x5f, 0x5f, 0x69, 0x64, 0x5f, 0x31], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "rotor_1__id_4" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x6f, 0x72, 0x5f, 0x31, 0x5f, 0x5f, 0x69, 0x64, 0x5f, 0x34], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "base_link__id_2" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x6c, 0x69, 0x6e, 0x6b, 0x5f, 0x5f, 0x69, 0x64, 0x5f, 0x32], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "rotor_0__id_5" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x6f, 